In [1]:
import sys
sys.path.append('..')

# Data Loading

In [2]:
from pathlib import Path
import pandas as pd

def load_datasets(data_root: str | Path = "data",
                  tasks: tuple[str, ...] = ("binary", "multiclass"),
                  splits: tuple[str, ...] = ("train", "val", "test")) -> dict:

    data_root = Path(data_root)
    datasets  = {}

    for task in tasks:
        task_dir     = data_root / task
        task_dict    = {}

        for split in splits:
            split_dict = {}
            for kind in ("X", "y"):
                file_path = task_dir / f"{kind}_{split}.pkl"
                split_dict[kind] = pd.read_pickle(file_path)
            task_dict[split] = split_dict

        datasets[task] = task_dict

    return datasets


# ── usage ────────────────────────────────────────────────────────
datasets = load_datasets("../data")

# quick sanity-check
for task, splits in datasets.items():
    for split, obj in splits.items():
        print(f"{task:<10} {split:<5}  X shape = {obj['X'].shape},  y len = {len(obj['y'])}")

binary     train  X shape = (4336, 2),  y len = 4336
binary     val    X shape = (1860, 2),  y len = 1860
binary     test   X shape = (1714, 2),  y len = 1714
multiclass train  X shape = (3237, 2),  y len = 3237
multiclass val    X shape = (1389, 2),  y len = 1389
multiclass test   X shape = (1332, 2),  y len = 1332


# Experiments

In [3]:
MODEL_TYPE = "LogReg"
SEED = 42

In [4]:
def get_datasets(kind: str):
    dataset = datasets[kind]
    
    X_train = dataset["train"]["X"]
    y_train = dataset["train"]["y"]
    
    X_val = dataset["val"]["X"]
    y_val = dataset["val"]["y"]
    
    X_test = dataset["test"]["X"]
    y_test = dataset["test"]["y"]

    return X_train, y_train, X_val, y_val, X_test, y_test

## Binary Classification

In [5]:
from utils import ExperimentManager, Experiment

CLASSIFICATION_TYPE = "binary"

manager = ExperimentManager(f"../runs/{CLASSIFICATION_TYPE}/{MODEL_TYPE}")
X_train, y_train, X_val, y_val, X_test, y_test = get_datasets(CLASSIFICATION_TYPE)

### Baseline with default scikit learn parameters and no optimization

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

def combine_text(X):
    X = X.copy() 

    combined = X["resume_text"].astype(str) + " [SEP] " + X["job_description_text"].astype(str)

    return combined.values

# Pipeline
pipe = Pipeline([
    ("join", FunctionTransformer(combine_text, validate=False)),
    ('tfidf', TfidfVectorizer()),  # Convert text to numeric
    ('clf', LogisticRegression(random_state=42))  # Logistic Regression model
])

def pipeline_factory(params):
    # Since we’re not using params here, we just return the static pipeline
    return pipe

experiment = Experiment(
    name=f"Baseline {CLASSIFICATION_TYPE} {MODEL_TYPE}",
    description=f"{CLASSIFICATION_TYPE} {MODEL_TYPE} with TF-IDF and no hyperparameter tuning.",
    pipeline_factory=pipeline_factory,
    splits=[(X_val, y_val), (X_test, y_test)],
    split_names=["Validation", "Test"]
)

manager.run_experiment(experiment, train_data=(X_train, y_train))


=== Running Experiment: Baseline binary LogReg ===

--- Validation Evaluation ---
Accuracy: 0.6812
Macro F1: 0.6802
Micro F1: 0.6812

Detailed Classification Report:
                 precision     recall   f1-score    support

       Good Fit     0.6628     0.7376     0.6982        930
         No Fit     0.7042     0.6247     0.6621        930

       accuracy                           0.6812       1860
      macro avg     0.6835     0.6812     0.6802       1860
   weighted avg     0.6835     0.6812     0.6802       1860

Confusion Matrix:
[[686 244]
 [349 581]]

--- Test Evaluation ---
Accuracy: 0.6004
Macro F1: 0.6000
Micro F1: 0.6004

Detailed Classification Report:
                 precision     recall   f1-score    support

       Good Fit     0.5951     0.6278     0.6110        857
         No Fit     0.6062     0.5729     0.5891        857

       accuracy                           0.6004       1714
      macro avg     0.6007     0.6004     0.6000       1714
   weighted avg   

## Multiclass Classification

In [7]:
from utils import ExperimentManager, Experiment

CLASSIFICATION_TYPE = "multiclass"

manager = ExperimentManager(f"../runs/{CLASSIFICATION_TYPE}/{MODEL_TYPE}")
X_train, y_train, X_val, y_val, X_test, y_test = get_datasets(CLASSIFICATION_TYPE)

### Basline with default scikit learn parameters and no optimization

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

def combine_text(X):
    X = X.copy() 

    combined = X["resume_text"].astype(str) + " [SEP] " + X["job_description_text"].astype(str)

    return combined.values

# Pipeline
pipe = Pipeline([
    ("join", FunctionTransformer(combine_text, validate=False)),
    ('tfidf', TfidfVectorizer()),  # Convert text to numeric
    ('clf', LogisticRegression(random_state=42))  # Logistic Regression model
])

def pipeline_factory(params):
    # Since we’re not using params here, we just return the static pipeline
    return pipe

experiment = Experiment(
    name=f"Baseline {CLASSIFICATION_TYPE} {MODEL_TYPE}",
    description=f"{CLASSIFICATION_TYPE} {MODEL_TYPE} with TF-IDF and no hyperparameter tuning.",
    pipeline_factory=pipeline_factory,
    splits=[(X_val, y_val), (X_test, y_test)],
    split_names=["Validation", "Test"]
)

manager.run_experiment(experiment, train_data=(X_train, y_train))


=== Running Experiment: Baseline multiclass LogReg ===

--- Validation Evaluation ---
Accuracy: 0.6048
Macro F1: 0.5994
Micro F1: 0.6048

Detailed Classification Report:
                 precision     recall   f1-score    support

       Good Fit     0.6341     0.7559     0.6897        463
         No Fit     0.5991     0.5875     0.5932        463
  Potential Fit     0.5692     0.4708     0.5154        463

       accuracy                           0.6048       1389
      macro avg     0.6008     0.6048     0.5994       1389
   weighted avg     0.6008     0.6048     0.5994       1389

Confusion Matrix:
[[350  60  53]
 [116 218 129]
 [ 86 105 272]]

--- Test Evaluation ---
Accuracy: 0.4324
Macro F1: 0.4317
Micro F1: 0.4324

Detailed Classification Report:
                 precision     recall   f1-score    support

       Good Fit     0.4559     0.4077     0.4304        444
         No Fit     0.3982     0.3964     0.3973        444
  Potential Fit     0.4442     0.4932     0.4674    